In [ ]:
import tweepy
import json
import requests
from nltk.corpus import stopwords # stopword examples, 'its', 'on', 'the', etc <---- will be helpful later
# most pythonistas will rename pandas as pd, numpy as np, and datetime as dt for short (you don't have to)
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
# open file with keys and set the path to your credentials JSON file
# you'll need to replace my file with yours
credentials = "keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [ ]:
bearer_token = api_tokens["bearer_token"]

In [ ]:
# we only want status info so we won't need the higher level access
client = tweepy.Client(bearer_token=bearer_token)

In [ ]:
tweets_rangers = client.search_recent_tweets(
    query = "Rangers -is:retweet",  # searches for #WPP while ignoring retweets
    max_results = 100
)

tweets_rangers.data

In [ ]:
ranger_tweets = []
for tweet in tweets_rangers.data:
    ranger_tweets.append(tweet.text)
    
ranger_tweets

In [ ]:
tweets_charles = client.search_recent_tweets(
    query = "King Charles -is:retweet",  # searches for #WPP while ignoring retweets
    max_results = 100
)

tweets_charles.data

In [ ]:
charles_tweets = []
for tweet in tweets_charles.data:
    charles_tweets.append(tweet.text)
    
charles_tweets

In [ ]:
import nltk
from nltk.corpus import stopwords

def get_features(text):
    # create array of words
    text = text.lower()
    words = text.split()
    s_words = stopwords.words('english')
    keywords = dict([(w, True) for w in words if w not in s_words and not w.isdigit() and '/' not in w])
    
    return keywords

get_features(charles_tweets[50])

In [ ]:
label_features = {'rangers': [], 'King Charles': []}

for tweet in ranger_tweets:
    features = get_features(tweet)
    label_features['rangers'].append(features)

In [ ]:
for tweet in charles_tweets:
    features = get_features(tweet)
    label_features['King Charles'].append(features)

In [ ]:
label_features['rangers'][0]

In [ ]:
import random
import math

# split the data into training / test sets -> 75/25% split
def split_label_feats(label_features):
    train_feats = []
    test_feats = []
    
    for label in label_features:
        features = label_features[label]
        cutoff = math.floor(len(features)*.8)
        train_feats.extend([(f, label) for f in features[:cutoff]])
        test_feats.extend([(f, label) for f in features[cutoff+1:]])

    return train_feats, test_feats

In [ ]:
train_set, test_set = split_label_feats(label_features)

In [ ]:
print(len(train_set))
print(len(test_set))

In [ ]:
# Naive Bayes Classifier
from nltk.classify import NaiveBayesClassifier

nb = NaiveBayesClassifier.train(train_set)

In [ ]:
ex1_features = get_features("Whatever he meant by it, he spoke for the country when he greeted Liz Truss. ‘Dear, oh dear’ just about sums things up.")
nb.classify(ex1_features)

In [ ]:
ex2_features = get_features("Haven't seen them fold as badly as that since, oh, about 2012")
nb.classify(ex2_features)

In [ ]:
# Calculate accuracy
from nltk.classify.util import accuracy

accuracy(nb, test_set)

In [ ]:
# getting classification probability of each label
probs = nb.prob_classify(test_set[6][0])

print(probs.samples())
print('chosen class:',probs.max())
print('P(rangers)=',probs.prob('rangers'))
print('P(King Charles)=',probs.prob('King Charles'))

In [ ]:
test_set[6]

In [ ]:
# more information on the chosen label

nb.show_most_informative_features(10)